In [1]:
# Librerias y dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoCV
from numpy import mean
from numpy import std
from numpy import arange
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.pipeline import Pipeline


In [2]:
# Clases y Funciones
'''
    Esta clase permite realizar la codificación en caliente especificameente variables categoricas
'''

class OneHotCoding():
    def __init__(self, df, bin_features):
        self.bin_features = bin_features
        self.df = df
    
    # Metodo para realizar la codificacion dummy a las variables categoricas

    def dummyCodification(self):
        cat_features = self.df.select_dtypes(include = ["object", "category"]).columns
        bin_dataset = self.df[self.bin_features].replace({'SI': 1, 'NO': 0})
        categorical_features = [x for x in cat_features if x not in self.bin_features]
        df_cat = pd.get_dummies(df[categorical_features])
        self.df.drop(cat_features, axis = 1, inplace = True)
        df_final = pd.concat([self.df,df_cat,bin_dataset ], axis = 1)
        df_final.to_excel("../Archivos Generados/PipelineResults/dasetOneHot.xlsx")
        print("Ejeción Terminada")
        return df_final


#categorical_transformer = Pipeline(
#    steps=[("OneHotCoding",  OneHotCoding(df,bin_features).dummyCodification())]
#)


class LinearRegession():
    def __init__(self, df, alpha, l1_ratio):
        self.df = df
        self.alpha = alpha
        self.l1_ratio = l1_ratio
    

    def CalcularModeloLR(self):
        # alpha=0.1, l1_ratio=0.97
        Y = self.df.RDT_AJUSTADO
        X = self.df.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1) 
        modelElasticNet = ElasticNet(alpha=self.alpha, l1_ratio=self.l1_ratio, random_state=123)
        model = modelElasticNet.fit(X,Y)
        r_2 = model.score(X,Y)
        # Pedicciones
        yhat = model.predict(X)

        return [model, r_2, yhat]


class CLR():

    def __init__(self, df, yhat):
        self.df = df
        self.yhat = yhat

    
    def calcularMAE(self):
        EPA = 0
        Acumulador = 0
        accepted_average_error= []
        self.df["yhat"]= pd.Series(self.yhat)
        self.df["EA"] = abs(self.df.RDT_AJUSTADO - self.df.yhat)
        self.df_ordely = self.df.sort_values(by=['EA'],ascending=True).reset_index()

        # Calculamos el MAE
        for i in range(len(self.df_ordely)):
            Acumulador = Acumulador + self.df_ordely.loc[i].EA
            EPA = Acumulador/(i+1)
            accepted_average_error.append(EPA)
        
        # Agregamos el promedio al dataset Ordenado.
        self.df_ordely["MAE"] = pd.Series(accepted_average_error)
        #self.df_ordely.to_excel("../Archivos Generados/PipelineResults/DatasetOrdenadoMAE.xlsx")
        #self.df.to_excel("../Archivos Generados/PipelineResults/DatasetOriginal.xlsx")
        return self.df_ordely
    


def DeleteRecordsGroup(Group, df):
    indexEliminar = list(Group["index"])
    df_new = df[df.index.isin(indexEliminar)== False]
    return df_new


# Retorna el grupo a modificar
def compareCorrelation(CorelacionGruposCalidad, nuevaCorrelation, listaGrupos):
    lista_grupos = listaGrupos
    #print("lista grupos: ",lista_grupos)
    arr =  np.array(CorelacionGruposCalidad) - np.array(nuevaCorrelation)
    position = np.where(arr == np.amin(arr))
    #print("Posision grupo: ",position)
    indexGrupoModificar = position[0][0]
    #print("index: ", indexGrupoModificar)
    lista_grupos.pop(indexGrupoModificar)
    return lista_grupos, indexGrupoModificar



In [3]:
# Dataset Original
df = pd.read_excel("../Archivos Generados/DatasetFinal.xlsx")
print(df.head())
# Ornial variables list
bin_features =['SEM_TRATADAS','DRENAJE','ALMACENAMIENTO_FINCA','CAP_ENDURE_RASTA','MOTEADOS_RASTA','MOTEADOS_MAS70cm._RASTA',
               'OBSERVA_EROSION_RASTA','OBSERVA_MOHO_RASTA','OBSERVA_RAICES_VIVAS_RASTA','OBSERVA_HOJARASCA_MO_RASTA',
                'SUELO_NEGRO_BLANDO_RASTA','CUCHILLO_PRIMER_HTE_RASTA','CERCA_RIOS_QUEBRADAS_RASTA',
               ]

dataset = OneHotCoding(df,bin_features).dummyCodification()

   ID_LOTE TIPO_SIEMBRA SEM_TRATADAS MATERIAL_GENETICO CULT_ANT DRENAJE  \
0       40   Mecanizado           NO     PIONEER 30F32  Algodon      SI   
1       43   Mecanizado           SI            DK 234     Maiz      SI   
2       44   Mecanizado           NO     PIONEER 30F32  Algodon      SI   
3       45   Mecanizado           NO              Otro  Algodon      SI   
4       46   Mecanizado           NO              Otro  Algodon      SI   

  METODO_COSECHA ALMACENAMIENTO_FINCA  DIAS_EN_EMERGER  \
0         Manual                   NO                5   
1         Manual                   NO                5   
2         Manual                   NO                5   
3         Manual                   NO                5   
4         Manual                   NO                5   

   DIAS_EN_EMERGER_A_FLORECER  ...  Temp_Max_Avg_Mad  Temp_Min_Avg_Mad  \
0                          63  ...             32.05             23.60   
1                          64  ...             32.37

In [12]:
name_columns = dataset.columns

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# transform data
dataset_normalizado = scaler.fit_transform(dataset)
print(dataset_normalizado)


[[0.00000000e+00 3.33333333e-01 6.27118644e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [6.83060109e-04 3.33333333e-01 6.44067797e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [9.10746812e-04 3.33333333e-01 5.59322034e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 ...
 [9.88160291e-01 4.00000000e-01 3.89830508e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [9.88615665e-01 4.66666667e-01 3.72881356e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.00000000e+00 2.66666667e-01 3.89830508e-01 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]]


In [18]:
len(dataset_normalizado[:][-1])

176

In [22]:
dataset_normalizado[:][0:-1]

In [13]:
import pandas as pd
dataset2 = pd.DataFrame(dataset_normalizado, columns=name_columns)
print(dataset2)

      ID_LOTE  DIAS_EN_EMERGER  DIAS_EN_EMERGER_A_FLORECER  \
0    0.000000         0.333333                    0.627119   
1    0.000683         0.333333                    0.644068   
2    0.000911         0.333333                    0.559322   
3    0.001138         0.333333                    0.644068   
4    0.001366         0.333333                    0.627119   
..        ...              ...                         ...   
794  0.987705         0.466667                    0.355932   
795  0.987933         0.333333                    0.355932   
796  0.988160         0.400000                    0.389831   
797  0.988616         0.466667                    0.372881   
798  1.000000         0.266667                    0.389831   

     DIAS_EN_FLORECER_A_COSECHAR  POBLACION_20DIAS_AJT  ALTURA_LOT  \
0                       0.328358              0.662577    0.142857   
1                       0.253731              0.662577    0.168831   
2                       0.298507             

In [14]:
print("Tamaño del dataset: ", dataset2.shape)
modellr, r_2, yhat = LinearRegession(dataset2, 0.1, 0.97).CalcularModeloLR()
print("Ajuste del Modelo dataset Completo: ", r_2)


Tamaño del dataset:  (799, 176)
Ajuste del Modelo dataset Completo:  0.0


In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

Y = dataset2.RDT_AJUSTADO
X = dataset2.drop(["RDT_AJUSTADO","ID_LOTE"], axis=1) 
modelElasticNet = ElasticNet(alpha=0.1, l1_ratio=0.97, random_state=123)
model = modelElasticNet.fit(X,Y)
r_2 = model.score(X,Y)
print(r_2)
yhat = model.predict(X)
prueba = r2_score(Y, yhat)
print(prueba)

0.0
0.0


In [90]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso, Ridge
from sklearn import linear_model
#reg = LinearRegression().fit(X, Y)
reg = Lasso().fit(X, Y)
reg2= Ridge().fit(X,Y)
reg3 = LinearRegression().fit(X,Y)
reg4 = ElasticNet().fit(X,Y)
print("Lasso: ", reg.score(X, Y))
print("Ridge: ", reg2.score(X,Y))
print("LR: ", reg3.score(X, Y))
print("Elasticnet: ", reg4.score(X,Y))


Lasso:  0.0
Ridge:  0.7496603412811106
LR:  0.760424034370608
Elasticnet:  0.0


In [54]:
Y

0      0.542751
1      0.525853
2      0.602737
3      0.561676
4      0.620481
         ...   
794    0.782697
795    0.661035
796    0.592093
797    0.687598
798    0.916864
Name: RDT_AJUSTADO, Length: 799, dtype: float64

In [32]:
Y

0      0.542751
1      0.525853
2      0.602737
3      0.561676
4      0.620481
         ...   
794    0.782697
795    0.661035
796    0.592093
797    0.687598
798    0.916864
Name: RDT_AJUSTADO, Length: 799, dtype: float64

In [30]:
dataset.shape

(799, 176)

In [31]:
Minimum_records = 80
minimum_correlation= 0.88
MAE_Allowed = 45
group_acepted = []
correlation_model =[]
model_acepted = []
additional_average_error = 50
contador = 0

In [41]:
def fase1(dataset):
    Minimum_records = 80
    minimum_correlation= 0.88
    MAE_Allowed = 45
    group_acepted = []
    correlation_model =[]
    model_acepted = []
    additional_average_error = 50
    contador = 0
    while (len(dataset) !=0):
        contador = contador+ 1
        print("Tamaño del dataset: ", dataset.shape)
        modellr, r_2, yhat = LinearRegession(dataset, 0.1, 0.97).CalcularModeloLR()
        print("Ajuste del Modelo dataset Completo: ", r_2)
        DatasetOrdely = CLR(dataset,yhat).calcularMAE()

        try:
            group = DatasetOrdely.loc[DatasetOrdely.MAE < MAE_Allowed]
        except:
             print(f"No se cumple con el criterio de selección, verificar variable MAE_ALLOWED: {MAE_Allowed}")
             # Retornar variables vacias
             break

        print(f"Longitud de grupo {contador}: ", len(group))
        if (len(group) >= Minimum_records):
                print("El grupo cumple minimo de registros")
                group = group.drop(["yhat","EA","MAE"], axis=1)
                dataset = dataset .drop(["yhat","EA"], axis=1)
                group_model, r2_group_mode, yhat_group = LinearRegession(group,0.1,0.97).CalcularModeloLR()
                print(f"R2 del grupo {contador}: ",r2_group_mode)
                if(r2_group_mode >= minimum_correlation):
                    # Elimino los registros para la siguiente iteración
                    dataset = DeleteRecordsGroup(group, dataset)
                    group = group.drop(['index'], axis=1)
                    group_acepted.append(group)
                    model_acepted.append(group_model)
                    correlation_model.append(r2_group_mode)
                    group.to_excel(f"R/Grupo_{contador}.xlsx")
                    MAE_Allowed = MAE_Allowed + additional_average_error
                else:
                    print("No cumple con la condición de  la correlacion")
                    Orphans = dataset
                    # Se eliminan las variables de trtamiento [yhat, EA]
                    Orphans = Orphans.drop(["yhat", "EA"],axis=1).reset_index(drop=True)
                    print("Logitud Huerfanos: ", Orphans.shape)
                    break  
        else:

            Orphans = dataset
            # Se eliminan las variables de trtamiento [yhat, EA]
            Orphans = Orphans.drop(["yhat", "EA"],axis=1).reset_index(drop=True)
            print("Logitud Huerfanos: ", Orphans.shape)
            # Se guardan los huerfanos en un archivo.
            Orphans.to_excel("R/Huerfanos.xlsx")
            break
    
    return[group_acepted, model_acepted, correlation_model, Orphans]

In [42]:
def fase2(group_acepted, correlation_model, Orphans):
    # Variables de entrada
    quality_groups = group_acepted.copy()
    correlation_quality_groups = correlation_model.copy()
    new_correlation = []

    group_list = list(range(len(quality_groups)))
    for register in range(len(Orphans)):
        for group in range(len(quality_groups)):
            quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)        
            new_model, new_r2, new_yhat = LinearRegession(quality_groups[group],0.1, 0.97).CalcularModeloLR()
            new_correlation.append(new_r2)

        # Compración de las correlaciones
        list_groups_remove, index = compareCorrelation(correlation_quality_groups, new_correlation, group_list)
        correlation_quality_groups[index] = new_correlation[index]
        new_correlation = []
        group_list = list(range(len(quality_groups)))

        # Eliminacion de Regristro en los demas grupos
        for i in list_groups_remove:
            quality_groups[i].drop([len(quality_groups[i]) -1 ],axis=0, inplace=True)

        Orphans.drop([0],axis=0, inplace=True)
        Orphans = Orphans.reset_index(drop=True)

    return [quality_groups,correlation_quality_groups, Orphans]

In [43]:
group_acepted, model_acepted, correlation_model, Orphans = fase1(dataset)

Tamaño del dataset:  (799, 176)


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.906e+07, tolerance: 1.533e+05
  model = cd_fast.enet_coordinate_descent(


Ajuste del Modelo dataset Completo:  0.7420467584701329
Longitud de grupo 1:  82
El grupo cumple minimo de registros
R2 del grupo 1:  0.9984019440207271


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.784e+05, tolerance: 7.284e+03
  model = cd_fast.enet_coordinate_descent(


Tamaño del dataset:  (717, 176)


...

Longitud de grupo 3:  87
El grupo cumple minimo de registros
R2 del grupo 3:  0.9930874180899754


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.038e+05, tolerance: 8.809e+03
  model = cd_fast.enet_coordinate_descent(


Tamaño del dataset:  (546, 176)
Ajuste del Modelo dataset Completo:  0.7586428604562752


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.002e+07, tolerance: 1.266e+05
  model = cd_fast.enet_coordinate_descent(


Longitud de grupo 4:  57
Logitud Huerfanos:  (546, 176)


In [44]:
len(Orphans)

546

In [45]:
quality_groups, correlation_quality_groups, orphans = fase2(group_acepted,correlation_model, Orphans)

C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.217e+05, tolerance: 7.365e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\s

[0.9983731059668111, 0.9823222011296522, 0.9922414694217224]
[0.9983820877581946, 0.9816209518993968, 0.9905011130674355]


C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.347e+05, tolerance: 7.472e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\s

[0.9983993635574246, 0.9821724543012058, 0.9930825436389431]
[0.9981550097308809, 0.9804303371717451, 0.9926318300553397]


C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.474e+05, tolerance: 7.474e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\s

...

c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.699e+06, tolerance: 9.097e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.448e+06, tolerance: 1.027e+04
  model = cd_fa

[0.8984679038111548, 0.9754424878612898, 0.9284900459457452]


C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.635e+06, tolerance: 9.106e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\s

[0.8990194073069048, 0.9813420536940693, 0.9280887980251675]
[0.8995441127926721, 0.9830639616250663, 0.9233252643903728]


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.622e+06, tolerance: 9.173e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\2321798219.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.148e+06, tolerance: 1.090e+04
  model = cd_fa

In [38]:
print(f"Grupo 1 {len(quality_groups[0])}, Grupo 2: {len(quality_groups[1])}, Grupo 3: {len(quality_groups[2])}")
print("correlaciones: ", correlation_quality_groups)

Grupo 1 398, Grupo 2: 84, Grupo 3: 317
correlaciones:  [0.8995441127926721, 0.989240743541881, 0.9284900459457452]


In [25]:
correlation_quality_groups

[0.9982753436624624, 0.9903962732759827, 0.9936473081024012]

In [12]:
print("Correlaciones: ", correlation_model)
print("Dimenciones de los huerfanos: ", Orphans.shape)

definitive_groups = []
if len(group_acepted) > 1:
    if len(Orphans) == 0:
        definitive_groups = group_acepted
    
    else:
        # Fase 2, incluir los huerfanos
        print("FASE 2")
        quality_groups, correlation_quality_groups, orphans = fase2(group_acepted,correlation_model, Orphans)

        print(len(orphans))
        print(correlation_quality_groups)


else:
    definitive_groups = Orphans



Correlaciones:  [0.9984019440207271, 0.9892407435418807, 0.9930874180899752]
Dimenciones de los huerfanos:  (546, 176)
FASE 2


C:\Users\germanm\AppData\Local\Temp\ipykernel_10416\3281268031.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [22]:
# Entradas Fase 2
quality_groups = group_acepted.copy()
correlation_quality_groups = correlation_model.copy()


print(f"Grupo Calidad 1: {len(quality_groups[0])} , Grupo de Calidad 2: {len(quality_groups[1])}, Grupo de Calidad 3: {len(quality_groups[2])}")
print("Correlaciones Grupos de calidad: ", correlation_quality_groups)


Grupo Calidad 1: 86 , Grupo de Calidad 2: 85, Grupo de Calidad 3: 83
Correlaciones Grupos de calidad:  [0.9982753436624624, 0.9903962732759827, 0.9936473081024012]


In [19]:
# Se requiere Buscar el mejor grupo
new_correlation = []
group_list = list(range(len(quality_groups)))
for register in range(len(Orphans)):
    for group in range(len(quality_groups)):
        quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)        
        new_model, new_r2, new_yhat = LinearRegession(quality_groups[group],0.1, 0.97).CalcularModeloLR()
        new_correlation.append(new_r2)

    
    #print("Nueva Correlacion: ", new_correlation)
    # Compración de las correlaciones
    list_groups_remove, index = compareCorrelation(correlation_quality_groups, new_correlation, group_list)
    correlation_quality_groups[index] = new_correlation[index]
    new_correlation = []
    group_list = list(range(len(quality_groups)))

    # Eliminacion de Regristro en los demas grupos
    for i in list_groups_remove:
        quality_groups[i].drop([len(quality_groups[i]) -1 ],axis=0, inplace=True)

    Orphans.drop([0],axis=0, inplace=True)
    Orphans = Orphans.reset_index(drop=True)

        

C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.457e+05, tolerance: 9.623e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\sit

Nueva Correlacion:  [0.9982104805861187, 0.9880618049506334, 0.9927450353656625]
Nueva Correlacion:  [0.9980200902195826, 0.9881638511770304, 0.9938535984513562]


C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.059e+05, tolerance: 8.216e+03
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\sit

Nueva Correlacion:  [0.9977554621103216, 0.9841371897126734, 0.9924179010928448]
Nueva Correlacion:  [0.9976806065110533, 0.9874513140355097, 0.9936426714141422]


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.969e+06, tolerance: 1.103e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.185e+05, tolerance: 8.226e+03
  model = cd_fas

...

C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.055e+07, tolerance: 6.404e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\sit

Nueva Correlacion:  [0.9211990507392728, 0.9403062241490614, 0.9543640299986136]


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.045e+07, tolerance: 6.467e+04
  model = cd_fast.enet_coordinate_descent(
C:\Users\germanm\AppData\Local\Temp\ipykernel_5004\3182360602.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  quality_groups[group] = quality_groups[group].append(Orphans.loc[0], ignore_index = True)
c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.634e+07, tolerance: 4.572e+04
  model = cd_fas

Nueva Correlacion:  [0.9195510487215257, 0.942195797570874, 0.9564233594833859]


c:\Users\germanm\.conda\envs\evertecenv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.914e+06, tolerance: 4.325e+04
  model = cd_fast.enet_coordinate_descent(


In [48]:
print(f"Grupo Calidad 1: {len(quality_groups[0])} , Grupo de Calidad 2: {len(quality_groups[1])}, Grupo de Calidad 3: {len(quality_groups[2])}")
print("Correlacion grupos finales: ", correlation_quality_groups)
print("Longitud de los huerfanos: ", len(orphans))


Grupo Calidad 1: 398 , Grupo de Calidad 2: 84, Grupo de Calidad 3: 317
Correlacion grupos finales:  [0.8995441127926721, 0.989240743541881, 0.9284900459457452]
Longitud de los huerfanos:  0
